In [9]:
import numpy as np
import os
import re
os.chdir("./Desktop/ArticleClassifier")

/Users/jackyteoh/Desktop/ArticleClassifier


In [10]:
# Run through 70% of the text files from each category (folder)
# 70% Training, 30% Testing for each category
#Creates training_titles and testing_tiles
training_titles_files = []
testing_titles_files = [] 

business_files = os.listdir("./business")
train_business = business_files[0:358]
test_business = business_files[358:] # length = 152 TOTAL = 510 
training_titles_files.append(train_business)
testing_titles_files.append(test_business)

entertainment_files = os.listdir("./entertainment")
train_entertainment = entertainment_files[0:270]
test_entertainment = entertainment_files[270:] # 116 TOTAL = 386
training_titles_files.append(train_entertainment)
testing_titles_files.append(test_entertainment)

politics_files = os.listdir("./politics")
train_politics = politics_files[0:293]
test_politics = politics_files[293:] # 124 # 417
training_titles_files.append(train_politics)
testing_titles_files.append(test_politics)

sport_files = os.listdir("./sport")
train_sport = sport_files[0:359]
test_sport = sport_files[359:] # 152 # 511
training_titles_files.append(train_sport)
testing_titles_files.append(test_sport)

tech_files = os.listdir("./tech")
train_tech = tech_files[0:281]
test_tech = tech_files[281:] # 120 #401
training_titles_files.append(train_tech)
testing_titles_files.append(test_tech)

In [12]:
training_text_file = open("trainingTextFile.txt", "w")

classes = 1
for i in training_titles_files:
	folder_name = ""
	if (classes == 1): 
		folder_name = "business"
	elif (classes == 2):
		folder_name = "entertainment"
	elif (classes == 3):
		folder_name = "politics"
	elif (classes == 4):
		folder_name = "sport"
	elif (classes == 5):
		folder_name = "tech"
	for file_name in i:
		file = "./" + folder_name + "/" + file_name
		#print(file)
		opened = open(file, "r")
		training_text_file.write(opened.readline())
		opened.close()
	classes = classes + 1

training_text_file.close()

masterTestDataFile = open("testTextFile.txt", "w")

classes = 1
for i in testing_titles_files:
	folder_name = ""
	if (classes == 1): 
		folder_name = "business"
	elif (classes == 2):
		folder_name = "entertainment"
	elif (classes == 3):
		folder_name = "politics"
	elif (classes == 4):
		folder_name = "sport"
	elif (classes == 5):
		folder_name = "tech"
	for file_name in i:
		file = "./" + folder_name + "/" + file_name
		#print (file)
		opened = open(file, "r")
		lineRead = opened.readline()
		#try: 
		masterTestDataFile.write(lineRead)
		#except UnicodeDecodeError:
		opened.close()
	classes = classes + 1

masterTestDataFile.close()

#totalcount = 0
#for i in testing_titles_files:
#	totalcount += len(i)

#print(totalcount)

#currency stemming nlp

In [13]:
training = []
testing = []

# Puts words in dictionary, key = word, value = index
trfile = open("trainingTextFile.txt", "r")

for line in trfile:
	line = line.replace("'", "")
	new_line = line.split()
	for i in new_line:
		training.append(i)

trfile.close()

words = {}
count = 0
for i in training:
	if i not in words:
		words[i] = count
		count = count + 1

In [14]:
#Create Training Data Matrix
d = len(words) # Number of features (unique words) we have in the dictionary
# print(d)
print(d)
train_x = []

trfile = open("trainingTextFile.txt", "r")

for line in trfile:
	line = line.replace("'", "")
	word = line.split()
	arr = [0] * d
	for i in word:
		index = words[i]
		arr[index] = arr[index] + 1
	train_x.append(arr)
trfile.close()

3454


In [15]:
#Target Training Vector
train_y = [0] * 1561
for i in range(358, 1561):
	if i >= 358 and i < 628:
		train_y[i] = 1
	elif i >= 628 and i < 921:
		train_y[i] = 2
	elif i >= 921 and i < 1280:
		train_y[i] = 3
	elif i >= 1280 and i < 1561:
		train_y[i] = 4

In [16]:
#Create Testing Data Matrix
test_x = []

testfile = open("testTextFile.txt", "r")

for line in testfile:
	line = line.replace("'", "")
	word = line.split()
	arr = [0] * d
	for i in word:
		if i in words:
			index = words[i]
			arr[index] = arr[index] + 1
	test_x.append(arr)
testfile.close()

In [17]:
#Target Test Vector
test_y = [0] * 664
for i in range(152, 664):
	if i >= 152 and i < 268:
		test_y[i] = 1
	elif i >= 268 and i < 392:
		test_y[i] = 2
	elif i >= 392 and i < 544:
		test_y[i] = 3
	elif i >= 544 and i < 664:
		test_y[i] = 4

countzero = 0
countone = 0
counttwo = 0
countthree = 0
countfour = 0
for i in test_y:
	if i == 0:
		countzero+=1
	if i == 1:
		countone += 1
	if i == 2:
		counttwo+=1 
	if i == 3:
		countthree+=1
	if i == 4:
		countfour +=1

print(countzero)
print(countone)
print(counttwo)
print(countthree)
print(countfour)


# y = 0 --> Business
# y = 1 --> Entertainment
# y = 2 --> Politics
# y = 3 --> Sport
# y = 4 --> Tech

152
116
124
152
120


In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

Using TensorFlow backend.


In [21]:
# epoch how many big loops
# batch = how many times each loop

# First model (smallest hidden layers (2))
model = Sequential()

model.add(Dense(units = d))

model.add(Dense(units = 500, activation = 'relu'))
model.add(Dense(units = 500, activation = 'relu'))

model.add(Dense(units = 5, activation = 'softmax'))

model.summary()

ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

In [22]:
# Second model (Medium hidden layers (4))
model = Sequential()

model.add(Dense(units = d))

model.add(Dense(units = 500, activation = 'relu'))
model.add(Dense(units = 500, activation = 'relu'))
model.add(Dense(units = 500, activation = 'relu'))
model.add(Dense(units = 500, activation = 'relu'))

model.add(Dense(units = 5, activation = 'softmax'))


In [23]:
# Third model (Large hidden layers (6))
model = Sequential()

model.add(Dense(units = d))

model.add(Dense(units = 500, activation = 'relu'))
model.add(Dense(units = 500, activation = 'relu'))
model.add(Dense(units = 500, activation = 'relu'))
model.add(Dense(units = 500, activation = 'relu'))
model.add(Dense(units = 500, activation = 'relu'))
model.add(Dense(units = 500, activation = 'relu'))

model.add(Dense(units = 5, activation = 'softmax'))